# Final Project
## By: Philip Spencer and Paul Jojy

# Exploring the effects of Quantization on Deep Learning
In this project, we will be exploring the effects of quantization on deep learning. This is an important topic in this field, as deep learning models are very large and can be inefficient to run on edge devices such as smart phones. ADD MORE LATER
    
## 1. Download and Preprocess Data
We will be analyzing CIFAR-10 and MNIST for this experiment. The data must be downloaded, normalized, and augmented to maximize performance.

In [1]:
#Required packages. Uncomment below lines if not in your system
#pip install torch
#pip install torchvision

In [20]:
import torch
from torchvision import models, datasets, transforms

def datadownload():
    cifar_augmenter = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ])
    
    cifar_train = datasets.CIFAR10('.',download=True, transform=cifar_augmenter)
    cifar_test = datasets.CIFAR10('.',train=False, transform=cifar_augmenter)
    
    mnist_augmenter = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(num_output_channels=3), #Convert grayscale to RGB where R=G=B. This is because the model is pretrained on an RGB dataset (Imagenet)
        transforms.ToTensor()
    ])
    
    mnist_train = datasets.MNIST('.',download=True, transform=mnist_augmenter)
    mnist_test = datasets.MNIST('.',train=False, transform=mnist_augmenter)
    return (cifar_train,cifar_test),(mnist_train,mnist_test)
cifar,mnist = datadownload()

Files already downloaded and verified


In [21]:
cifar

(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: .
     Split: Train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            ),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: .
     Split: Test
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            ))

In [22]:
mnist

(Dataset MNIST
     Number of datapoints: 60000
     Root location: .
     Split: Train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                Grayscale(num_output_channels=3)
                ToTensor()
            ),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: .
     Split: Test
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                Grayscale(num_output_channels=3)
                ToTensor()
            ))